In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from utils_test import *

In [2]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [3]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [4]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_77-1.51.h5',
}

args = argparse.Namespace(**args)

In [5]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [6]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [7]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [8]:
model = load_model(args.outputdir+args.modelname)

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


In [9]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [10]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix and next_chars[i] != 'EOS':
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

In [11]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print 'Log-loss: {}'.format(log_loss)
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print 'Accuracy: {}%'.format(acc*100)
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print 'Top 3 accuracy: {}%'.format(acc_3*100)

In [12]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print 'Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400)
    print 'Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400)
    print 'Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400)

In [13]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

# Evaluation

In [14]:
max_suffix = 11

In [15]:
for i in range(2, max_suffix+1):
    print '----------Suffix {}------------'.format(i)
    sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 = getSuffix(i, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)
    X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)
    pred = model.predict(X_test, verbose=0)
    pred_a = pred[0]
    pred_t = pred[1]
    print 'Evaluation of Activity'
    evalAct(next_chars_2, pred[0], targetchartoindice)
    print 'Evaluation of Time'
    evalTime(next_chars_t_2, pred_t, divisor)
    print '\n'
    #Add gt and pred of each suffix
    if i == 2:
        full_gt_a = next_chars_2
        full_pred_a = pred_a
        full_gt_t = next_chars_t_2
        full_pred_t = pred_t
    else:
        full_gt_a += next_chars_2
        full_pred_a = np.concatenate((full_pred_a, pred_a), axis=0)
        full_gt_t += next_chars_t_2
        full_pred_t = np.concatenate((full_pred_t, pred_t), axis=0)

----------Suffix 2------------
Evaluation of Activity
Log-loss: 0.686563246418
Accuracy: 76.9166666667%
Top 3 accuracy: 98.6666666667%
Evaluation of Time
Mean Squared Error: 3.68835949788e+11s	| 4268934.60403 days
Mean Absolute Error: 293178.6852s	| 3.39327181945 days
Median Absolute Error: 30281.7763672s	| 0.350483522768 days


----------Suffix 3------------
Evaluation of Activity
Log-loss: 1.19477937787
Accuracy: 57.5280898876%
Top 3 accuracy: 98.4269662921%
Evaluation of Time
Mean Squared Error: 5.27921909351e+11s	| 6110207.28416 days
Mean Absolute Error: 416259.006823s	| 4.81781257897 days
Median Absolute Error: 209335.1875s	| 2.42286096644 days


----------Suffix 4------------
Evaluation of Activity
Log-loss: 1.00470066233
Accuracy: 65.1282051282%
Top 3 accuracy: 98.9743589744%
Evaluation of Time
Mean Squared Error: 2.72850015647e+11s	| 3157986.2922 days
Mean Absolute Error: 227371.455346s	| 2.6316140665 days
Median Absolute Error: 11786.8066406s	| 0.136421373155 days


----------

In [16]:
evalAct(full_gt_a, full_pred_a, targetchartoindice)

Log-loss: 0.844332611073
Accuracy: 70.9984947316%
Top 3 accuracy: 98.6452584044%


In [17]:
evalTime(full_gt_t, full_pred_t, divisor)

Mean Squared Error: 3.97393093724e+11s	| 4599457.10328 days
Mean Absolute Error: 312937.517781s	| 3.62196201136 days
Median Absolute Error: 39017.71875s	| 0.451593967014 days


# Test single case

In [18]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(10, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [19]:
len(sentences_2), len(next_chars_2), len(sentences)

(3, 3, 4529)

In [20]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [21]:
pred = model.predict(X_test, verbose=0)
pred_a1 = pred[0]
pred_t = pred[1]

In [22]:
pred[0].shape, pred[1].shape

((3, 10), (3, 1))

## Evaluate

### Activity

In [23]:
evalAct(next_chars_2, pred[0], targetchartoindice)

Log-loss: 0.156090296494
Accuracy: 100.0%
Top 3 accuracy: 100.0%


### Time

In [24]:
evalTime(next_chars_t_2, pred_t, divisor)

Mean Squared Error: 80774949448.4s	| 934895.248246 days
Mean Absolute Error: 215542.325734s	| 2.49470284415 days
Median Absolute Error: 193860.84375s	| 2.24375976563 days
